In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
import pandas as pd
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

from train_models import ClassifierTrainer
from text_utils import normalize_text
from callbacks import EarlyStoppingCallback, ReduceLROnPlateauCallback, LRSchedulerCallback

/Users/aleksejkitajskij/Library/Mobile Documents/com~apple~CloudDocs/Programing/GoIT_projects/deep learning/final_DL/train_models.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
DEVICE = 'cpu'
BATCH_SIZE = 8
NUM_CLASSES = 4
IMG_SIZE = 224

In [3]:
df = pd.read_csv("data/train.csv")

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>прописую шлях до всіх фото в датафрейм </b></div>

берется тільки перше фото аби не ускладнювати код

In [4]:
path_to_images = "data/images/train/"
df['image_path'] = path_to_images + df['PetID'].astype(str) + "-1.jpg"

In [5]:
df

,PetID,Description,AdoptionSpeed,image_path
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2,data/images/train/d3b4f29f8-1.jpg
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2,data/images/train/e9dc82251-1.jpg
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2,data/images/train/8111f6d4a-1.jpg
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2,data/images/train/693a90fda-1.jpg
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2,data/images/train/9d08c85ef-1.jpg
...,...,...,...,...
6426,440676acb,Puppy was rescued in the monsoon drain near my...,2,data/images/train/440676acb-1.jpg
6427,62550cf78,Please contact if you are interested.,1,data/images/train/62550cf78-1.jpg
6428,a062234cb,4/1/ An adorable domestic medium hair kitten l...,4,data/images/train/a062234cb-1.jpg
6429,7a54b760e,6 Jan :30: We found this poor dog (male) wande...,2,data/images/train/7a54b760e-1.jpg


<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>нормалізуєтся текст</b></div>

нормалізація проходить стандартним шляхом: видаленя пунктуацій, стопслів, лемізація слів і т.д.

In [6]:
df['Description'] = df['Description'].apply(normalize_text)
df.sample(5)

,PetID,Description,AdoptionSpeed,image_path
4331,216840542,male female puppy waiting adoption ipoh area,4,data/images/train/216840542-1.jpg
2033,a6fc65f2d,find cat cry outside house neighbours yard hid...,4,data/images/train/a6fc65f2d-1.jpg
652,060d1de98,kldesa petale,4,data/images/train/060d1de98-1.jpg
6404,9beedceb9,simba playful friendly boy healthy condition i...,4,data/images/train/9beedceb9-1.jpg
5614,ccd14639f,rescue puppy adoption puppy free adoption pupp...,3,data/images/train/ccd14639f-1.jpg


In [7]:
le = LabelEncoder()
df['AdoptionSpeed'] = le.fit_transform(df['AdoptionSpeed'])
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5144 entries, 915 to 860
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PetID          5144 non-null   object
 1   Description    5144 non-null   object
 2   AdoptionSpeed  5144 non-null   int64 
 3   image_path     5144 non-null   object
dtypes: int64(1), object(3)
memory usage: 200.9+ KB


<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>підготовка трансформерів </b></div>

In [9]:
tfidf = TfidfVectorizer(max_features=500)
tfidf.fit(train_df['Description'])

img_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>підготовка батчів</b></div>

In [10]:
class PetDataset(Dataset):
    def __init__(self, df, tfidf, transform, is_test=False):
        self.df = df.reset_index(drop=True)
        self.tfidf = tfidf
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            img = Image.open(row['image_path']).convert('RGB')
        except FileNotFoundError:
            print(f"Image not found: {row['image_path']}")
            img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (255, 255, 255))
        img = self.transform(img)

        text = self.tfidf.transform([row['Description']]).toarray().squeeze()
        text = torch.tensor(text, dtype=torch.float32)

        sample = {'image': img, 'text': text}

        if not self.is_test:
            sample['label'] = torch.tensor(row['AdoptionSpeed'], dtype=torch.long)

        return sample


train_ds = PetDataset(train_df, tfidf, img_transforms)
val_ds = PetDataset(val_df, tfidf, img_transforms)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>побудова моделі </b></div>

In [11]:
resnet18_model = models.resnet18(weights='IMAGENET1K_V1')
resnet18_model.fc = nn.Identity()
x = torch.randn(1, 3, 224, 224) # рандомна матриця для проходження через модель з параметрами зображення
with torch.no_grad():
    features = resnet18_model(x)

pretreined_model_output_shape = features.shape[1]
pretreined_model_output_shape

512

In [12]:
class AdoptionModel(nn.Module):
    def __init__(self, text_dim=500, num_classes=4):
        super().__init__()
        self.cnn = resnet18_model

        self.text_net = nn.Sequential(
            nn.Linear(text_dim, 128),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(pretreined_model_output_shape + 128, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, inputs):
        image, text = inputs
        img_feat = self.cnn(image)
        txt_feat = self.text_net(text)
        x = torch.cat((img_feat, txt_feat), dim=1)
        return self.classifier(x)

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>налаштування трейнера </b></div>

використовуватимется мій кастомний трейнер, який я намагався зробити універсальним. проте оскільки в моделі буде відбуватись входження двох інпутів (тексту та зображення) необхідно перевизначити метод _parse_batch

In [13]:
class AdoptionTrainer(ClassifierTrainer):
    def _parse_batch(self, batch):
        images = batch['image'].to(self.device)
        texts = batch['text'].to(self.device)
        labels = batch['label'].to(self.device)
        return (images, texts), labels, None

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>тренування </b></div>

In [14]:
criterion = nn.CrossEntropyLoss()
model = AdoptionModel()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

trainer = AdoptionTrainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    num_classes=NUM_CLASSES,
    device=DEVICE
)

trainer.fit(train_loader, val_loader, num_epoch=1)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/1] Train Loss: 1.2959 Acc: 0.3541 Rec: 0.3541 Prec: 0.3639
Epoch [1/1] Val Loss: 1.2304 Acc: 0.4018 Rec: 0.4018 Prec: 0.4038


<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>прогнозування</b></div>

In [16]:
test = pd.read_csv("data/test.csv")

path_to_images = "data/images/test/"
test['image_path'] = path_to_images + test['PetID'].astype(str) + "-1.jpg"

test['Description'] = test['Description'].apply(normalize_text)

test_ds = PetDataset(test, tfidf, img_transforms, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [17]:
model.eval()
preds = []
with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(DEVICE)
        texts = batch['text'].to(DEVICE)
        outputs = model((images, texts))
        pred_labels = outputs.argmax(dim=1).cpu().numpy()
        preds.extend(pred_labels)

Image not found: data/images/test/95314294-1.jpg
Image not found: data/images/test/35992662-1.jpg
Image not found: data/images/test/63521459-1.jpg
Image not found: data/images/test/81301773-1.jpg


In [18]:
preds = le.inverse_transform(preds)
test['AdoptionSpeed'] = preds
test

,PetID,Description,image_path,AdoptionSpeed
0,6697a7f62,cute little puppy look love home smart lovable...,data/images/test/6697a7f62-1.jpg,4
1,23b64fe21,puppy rescue mechanic shop sentul apparently m...,data/images/test/23b64fe21-1.jpg,1
2,41e824cbe,ara need forever home believe he really sweet ...,data/images/test/41e824cbe-1.jpg,4
3,6c3d7237b,rescue homeless dog year ago parent allow keep...,data/images/test/6c3d7237b-1.jpg,3
4,97b0b5d92,find shopping mall clean state hi active hope ...,data/images/test/97b0b5d92-1.jpg,4
...,...,...,...,...
1886,986e26eeb,give stray kitten home may save life much want...,data/images/test/986e26eeb-1.jpg,2
1887,9b2316d19,reno affectionate playful boy smart alert frie...,data/images/test/9b2316d19-1.jpg,4
1888,c60193e34,spot active playful like run around play sibli...,data/images/test/c60193e34-1.jpg,1
1889,4f7a70728,hello give love warm forever home please conta...,data/images/test/4f7a70728-1.jpg,4


In [ ]:
test[['PetID', 'AdoptionSpeed']].to_csv("my_submission.csv", index=False)